Import libraries:

In [ ]:
!pip install essentia

     |████████████████████████████████| 11.7MB 339kB/s 


In [ ]:
#Basic imports
import os
import matplotlib.pyplot as plt
import PIL
import essentia.standard as ess
import essentia
import numpy as np
import IPython.display as ipd
from random import randint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
count = 0
count_songs = 0
labels = []
labels_dict = dict()


# iterate over directories in '../../'
# each directory is a label
# store the in labels variable
for root, dirs, files in os.walk ('/content/drive/My Drive/archives/mel_specs/1_min/train'):
    count += 1
    if count==1:
        labels = [label for label in dirs if not label.startswith('.')]
    else:
        break

for l in labels:
    # count the number of songs in labels directories
    # file is considered to be song if it ends with .mp3 and doesn't
    # start with .
    count_songs = 0
    for root, dirs, files in os.walk('/content/drive/My Drive/archives/mel_specs/1_min/train'+l):
        for file in files:
            if file.endswith('.npy'):
                count_songs += 1

    # store label and it's count in labels_dict["name"] and
    # labels_dict["number_songs"] lists
    print(l, " songs ", count_songs)
    if "name" in labels_dict:
        labels_dict["name"].append(l)
        labels_dict['number_songs'].append(count_songs)
    else:
        labels_dict["name"] = [l]
        labels_dict["number_songs"] = [count_songs]

suaraarchive  songs  0
stilvortalentarchive  songs  0
ninjatunearchive  songs  0
mordarchive  songs  0
littlehelpersarchive  songs  0
kompaktarchive  songs  0
bar25archive  songs  0
defectedarchive  songs  0
chillhoparchive  songs  0


# Mel spectrogram calculation

Create folder to save mel spectrogram images

In [ ]:
img_dir_name = '/homedtic/jbustos/mel_specs'

if not os.path.exists(img_dir_name):
    os.mkdir(img_dir_name)
    print("Directory ", img_dir_name, " Created ")
else:
    print("Directory ", img_dir_name, " already exists")

### 1 minute chunk 

Create new directory to save mel spectrograms of 1 minute chunk

In [ ]:
minute_1_folder = '/homedtic/jbustos/mel_specs/1_minute_BoC'

if not os.path.exists(minute_1_folder):
    os.mkdir(minute_1_folder)
    print("Directory ", minute_1_folder, " Created ")
else:
    print("Directory ", minute_1_folder, " already exists")

Directory  /homedtic/jbustos/mel_specs  already exists
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC  Created 


In [ ]:
countlabel = 0
count = 0
nfiles = min(labels_dict["number_songs"])
fs = 44100
sample_1m = fs*60
   
# Start iteration through labels folders
for l in labels_dict['name']:

    if not os.path.exists(minute_1_folder+'/'+l):
        os.mkdir(minute_1_folder+'/'+l)
        print("Directory ", minute_1_folder+'/'+l, " Created ")
    else:
        print("Directory ", minute_1_folder+'/'+l, " already exists")
        
        
    for root, dirs, files in os.walk (main_dir+l):
        # Check if counter of files per label and reset
        count = 0
        # Iterate over files in label l
        for file in files: 
            if not file.startswith('.') and file.endswith('.mp3'):                                  
                if count <= nfiles:
                    count += 1
                    # Reading audio file 
                    x = ess.MonoLoader(filename=root+'/'+file, sampleRate=fs)()
                    
                    # Append audio file choppend in a list
                    X = []
                    for i in range(x.size//sample_1m):
                        X.append(x[i*sample_1m:i*sample_1m+sample_1m])
                    
                    # Initilization of spectrum, window and melbands
                    spectrum = ess.Spectrum()
                    windowing = ess.Windowing(type='blackmanharris62', zeroPadding=2048)
                    melbands = ess.MelBands(numberBands=96, lowFrequencyBound=0, highFrequencyBound=11000)

                    pool = essentia.Pool()
                    amp2db = ess.UnaryOperator(type='lin2db', scale=2)
                    
                    # Mel band calculation for each chunk
                    count_chunk = 0
                    
                    # Create a folder for each song
                    if not os.path.exists(minute_1_folder+'/'+l+'/'+ file):
                        os.mkdir(minute_1_folder+'/'+l+'/'+file)
                        print("Directory ", minute_1_folder+'/'+l+'/'+file, " Created ")
                    else:
                        print("Directory ", minute_1_folder+'/'+l+'/'+file, " already exists")
    
                    for x in X:
                        count_chunk+=1;
                        pool = essentia.Pool()
                        for frame in ess.FrameGenerator(x, frameSize=2048, hopSize=1024):
                            frame_spectrum = spectrum(windowing(frame))
                            frame_mel = melbands(frame_spectrum)

                            pool.add('mel96_db', amp2db(frame_mel))                               
                           
                        plt.imsave(minute_1_folder+'/'+l+'/'+file+'/' + str(count_chunk) + '.jpg', pool['mel96_db'].T)   
                            
                    if count % 20 == 0:
                        print(count, "files processed, current file: ",file)                    



print('Files processed ', count)

Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Danny Serrano & Westboy - Bubblegun.mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Edu Imbernon & Triumph - Neim.mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Edu Imbernon & Triumph - Neim (Milton Jackson Remix).mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Ramiro Lopez - Filgud..mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Ramiro Lopez - Real Party (Martin Dawson Remix).mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Ramiro Lopez - Real Party..mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Ramiro Lopez - Filgud  (Coyu 'Feelraw' Remix).mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/03 DJ Kool Dek - DatBoombaplsh! (Soul Psycho

Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Coyu, Thomas Gandey - 1+1 Feat. Thomas Gandey (Truncate Remix) [Suara].mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Audiojack - Singularity (Original Mix) [Suara]-electrobuzz.net.mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Audiojack - Rendezvous (Original Mix) [Suara]-electrobuzz.net.mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Hidden Empire - First Bastion (Original Mix) [Suara].mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Hidden Empire - Weekend Warrior (Original Mix) [Suara].mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Kiko, Citizen Kain - Through And Through (Original Mix) [Suara].mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Kiko, Citizen Kain - Goon On The Moon (Original Mix) [Suara].mp3  Created 
Directory  /homedt

Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Hobo - Whiptrack (Charles Fenckler Interpretation) [Suara].mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Hobo - Whiptrack (Original Mix) [Suara].mp3  Created 
120 files processed, current file:  Hobo - Whiptrack (Original Mix) [Suara].mp3
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Mark Jenkyns - Reach For Me Feat. Mizbee (Richy Ahmed Remix) [Suara].mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Mark Jenkyns - Reach For Me Feat. Mizbee (Original Mix) [Suara].mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Mark Jenkyns - Reach For Me Feat. Mizbee (CamelPhat Remix) [Suara].mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Mark Jenkyns - Reach For Me Feat. Mizbee (Raw District Remix) [Suara].mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Loco & Jam -

Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Tube & Berger-Imprint Of Pleasure (Adam Beyer Remix) [Suara]-electrobuzz.mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Tube & Berger-Imprint Of Pleasure (Monkey Safari Remix) [Suara]-electrobuzz.mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Tube & Berger-Imprint Of Pleasure (Coyu Remix) [Suara]-electrobuzz.mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Tube & Berger-Imprint Of Pleasure (Rodriguez Jr. Remix) [Suara]-electrobuzz.mp3  Created 
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Sonic Future-L.o.v.e. (Original Mix)-electrobuzz.me.mp3  Created 
180 files processed, current file:  Sonic Future-L.o.v.e. (Original Mix)-electrobuzz.me.mp3
Directory  /homedtic/jbustos/mel_specs/1_minute_BoC/suaraarchive/Vario Volinski-Falling In Love (Jon Sine Remix) [Suara]-electrobuzz.mp3  Created 
Directory  /homedtic/j

In [ ]:
print(2646000/2048)

1291.9921875


In [ ]:
print(pool['mel96_db'].T.shape)

plt.imsave('/homedtic/jbustos/'+'outfile.jpg', pool['mel96_db'].T, origin='lower')

(96, 2585)


### 30 seconds chunk

Create folder for 30 seconds chunks

In [ ]:
seconds_30_folder = '/homedtic/jbustos/mel_specs/30_seconds_BoC'

if not os.path.exists(seconds_30_folder):
    os.mkdir(seconds_30_folder)
    print("Directory ", seconds_30_folder, " Created ")
else:
    print("Directory ", seconds_30_folder, " already exists")

In [ ]:
countlabel = 0
count = 0
nfiles = min(labels_dict["number_songs"])
fs = 44100
sample_30s = fs*30
   
# Start iteration through labels folders
for l in labels_dict['name']:

    if not os.path.exists(seconds_30_folder +'/'+l):
        os.mkdir(seconds_30_folder +'/'+l)
        print("Directory ", seconds_30_folder +'/'+l, " Created ")
    else:
        print("Directory ", seconds_30_folder +'/'+l, " already exists")
        
        
    for root, dirs, files in os.walk (main_dir+l):
        # Check if counter of files per label and reset
        count = 0
        # Iterate over files in label l
        for file in files: 
            if not file.startswith('.') and file.endswith('.mp3'):                                  
                if count <= nfiles:
                    count += 1
                    # Reading audio file 
                    x = ess.MonoLoader(filename=root+'/'+file, sampleRate=fs)()
                    
                    # Append audio file choppend in a list
                    X = []
                    for i in range(x.size//sample_30s):
                        X.append(x[i*sample_30s:i*sample_30s+sample_30s])
                    
                    # Initilization of spectrum, window and melbands
                    spectrum = ess.Spectrum()
                    windowing = ess.Windowing(type='blackmanharris62', zeroPadding=2048)
                    melbands = ess.MelBands(numberBands=96, lowFrequencyBound=0, highFrequencyBound=11000)

                    pool = essentia.Pool()
                    amp2db = ess.UnaryOperator(type='lin2db', scale=2)
                    
                    # Mel band calculation for each chunk
                    count_chunk = 0
                    
                    # Create a folder for each song
                    if not os.path.exists(seconds_30_folder+'/'+l+'/'+ file):
                        os.mkdir(seconds_30_folder+'/'+l+'/'+file)
                        print("Directory ", seconds_30_folder+'/'+l+'/'+file, " Created ")
                    else:
                        print("Directory ", seconds_30_folder+'/'+l+'/'+file, " already exists")
    
                    for x in X:
                        count_chunk+=1;
                        pool = essentia.Pool()
                        for frame in ess.FrameGenerator(x, frameSize=2048, hopSize=1024):
                            frame_spectrum = spectrum(windowing(frame))
                            frame_mel = melbands(frame_spectrum)

                            pool.add('mel96_db', amp2db(frame_mel))                               
                           
                        plt.imsave(seconds_30_folder+'/'+l+'/'+file+'/' + str(count_chunk) + '.jpg', pool['mel96_db'].T)   
                            
                    if count % 20 == 0:
                        print(count, "files processed, current file: ",file)                    


print('Files processed ', count)

### 15 seconds chunk

Create folder for 15 seconds chunks

In [ ]:
seconds_15_folder = '/homedtic/jbustos/mel_specs/15_seconds_BoC'

if not os.path.exists(seconds_15_folder):
    os.mkdir(seconds_15_folder)
    print("Directory ", seconds_15_folder, " Created ")
else:
    print("Directory ", seconds_15_folder, " already exists")

In [ ]:
countlabel = 0
count = 0
nfiles = min(labels_dict["number_songs"])
fs = 44100
sample_15s = fs*15
   
# Start iteration through labels folders
for l in labels_dict['name']:

    if not os.path.exists(seconds_15_folder +'/'+l):
        os.mkdir(seconds_15_folder +'/'+l)
        print("Directory ", seconds_15_folder +'/'+l, " Created ")
    else:
        print("Directory ", seconds_15_folder +'/'+l, " already exists")
        
        
    for root, dirs, files in os.walk (main_dir+l):
        # Check if counter of files per label and reset
        count = 0
        # Iterate over files in label l
        for file in files: 
            if not file.startswith('.') and file.endswith('.mp3'):                                  
                if count <= nfiles:
                    count += 1
                    # Reading audio file 
                    x = ess.MonoLoader(filename=root+'/'+file, sampleRate=fs)()
                    
                    # Append audio file choppend in a list
                    X = []
                    for i in range(x.size//sample_15s):
                        X.append(x[i*sample_15s:i*sample_15s+sample_15s])
                    
                    # Initilization of spectrum, window and melbands
                    spectrum = ess.Spectrum()
                    windowing = ess.Windowing(type='blackmanharris62', zeroPadding=2048)
                    melbands = ess.MelBands(numberBands=96, lowFrequencyBound=0, highFrequencyBound=11000)

                    pool = essentia.Pool()
                    amp2db = ess.UnaryOperator(type='lin2db', scale=2)
                    
                    # Mel band calculation for each chunk
                    count_chunk = 0
                    
                    # Create a folder for each song
                    if not os.path.exists(seconds_15_folder+'/'+l+'/'+ file):
                        os.mkdir(seconds_15_folder+'/'+l+'/'+file)
                        print("Directory ", seconds_15_folder+'/'+l+'/'+file, " Created ")
                    else:
                        print("Directory ", seconds_15_folder+'/'+l+'/'+file, " already exists")
    
                    for x in X:
                        count_chunk+=1;
                        pool = essentia.Pool()
                        for frame in ess.FrameGenerator(x, frameSize=2048, hopSize=1024):
                            frame_spectrum = spectrum(windowing(frame))
                            frame_mel = melbands(frame_spectrum)

                            pool.add('mel96_db', amp2db(frame_mel))                               
                           
                        plt.imsave(seconds_15_folder+'/'+l+'/'+file+'/' + str(count_chunk) + '.jpg', pool['mel96_db'].T)   
                            
                    if count % 20 == 0:
                        print(count, "files processed, current file: ",file)                    


print('Files processed ', count)

### 5 seconds chunk

In [ ]:
seconds_5_folder = '/homedtic/jbustos/mel_specs/5_seconds_BoC'

if not os.path.exists(seconds_5_folder):
    os.mkdir(seconds_5_folder)
    print("Directory ", seconds_5_folder, " Created ")
else:
    print("Directory ", seconds_5_folder, " already exists")

In [ ]:
countlabel = 0
count = 0
nfiles = min(labels_dict["number_songs"])
fs = 44100
sample_5s = fs*5
   
# Start iteration through labels folders
for l in labels_dict['name']:

    if not os.path.exists(seconds_5_folder +'/'+l):
        os.mkdir(seconds_5_folder +'/'+l)
        print("Directory ", seconds_5_folder +'/'+l, " Created ")
    else:
        print("Directory ", seconds_5_folder +'/'+l, " already exists")
        
        
    for root, dirs, files in os.walk (main_dir+l):
        # Check if counter of files per label and reset
        count = 0
        # Iterate over files in label l
        for file in files: 
            if not file.startswith('.') and file.endswith('.mp3'):                                  
                if count < nfiles:
                    count += 1
                    # Reading audio file 
                    x = ess.MonoLoader(filename=root+'/'+file, sampleRate=fs)()
                    
                    # Append audio file choppend in a list
                    X = []
                    for i in range(x.size//sample_5s):
                        X.append(x[i*sample_5s:i*sample_5s+sample_5s])
                    
                    # Initilization of spectrum, window and melbands
                    spectrum = ess.Spectrum()
                    windowing = ess.Windowing(type='blackmanharris62', zeroPadding=2048)
                    melbands = ess.MelBands(numberBands=96, lowFrequencyBound=0, highFrequencyBound=11000)

                    pool = essentia.Pool()
                    amp2db = ess.UnaryOperator(type='lin2db', scale=2)
                    
                    # Mel band calculation for each chunk
                    count_chunk = 0
                    
                    # Create a folder for each song
                    if not os.path.exists(seconds_5_folder+'/'+l+'/'+ file):
                        os.mkdir(seconds_5_folder+'/'+l+'/'+file)
                        print("Directory ", seconds_5_folder+'/'+l+'/'+file, " Created ")
                    else:
                        print("Directory ", seconds_5_folder+'/'+l+'/'+file, " already exists")
    
                    for x in X:
                        count_chunk+=1;
                        pool = essentia.Pool()
                        for frame in ess.FrameGenerator(x, frameSize=2048, hopSize=1024):
                            frame_spectrum = spectrum(windowing(frame))
                            frame_mel = melbands(frame_spectrum)

                            pool.add('mel96_db', amp2db(frame_mel))                               
                           
                        plt.imsave(seconds_5_folder+'/'+l+'/'+file+'/' + str(count_chunk) + '.jpg', pool['mel96_db'].T)   
                            
                    if count % 20 == 0:
                        print(count, "files processed, current file: ",file)                    


print('Files processed ', count)

# CNN implementation

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class CNN(nn.Modules):

  def __init__(self,):
    self.conv1 = nn.Conv2d(in_channels=1,out_channels, kernel_size=3,padding=1)